In [19]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
class Conv_block(nn.Module):
    def __init__(self,inp=3):
        super(Conv_block,self).__init__()
        self.conv1=nn.Conv2d(in_channels=inp,out_channels=64,kernel_size=3)
        self.conv2a=nn.Conv2d(in_channels=inp,out_channels=64,kernel_size=3,padding=(1,1))
        self.conv2b=nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3)
        self.conv3=nn.Conv2d(in_channels=inp,out_channels=64,kernel_size=3,dilation=2,padding=(1,1))
        self.instance_norm=nn.InstanceNorm2d(num_features=64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
        self.drop=nn.Dropout2d(p=0.1, inplace=False)
        self.pool=nn.MaxPool2d(2, stride=None, padding=0, dilation=1, return_indices=False, ceil_mode=False)
    
    def _forward(self,x):
        branch1=self.conv1(x)
        branch2=self.conv2a(x)
        branch2=self.conv2b(branch2)
        branch3=self.conv3(x)
        outputs = [branch1,branch2,branch3]
        return outputs
    
    def c_relu(self,x):
        x_m=x*-1
        ###CHECK WITH DIMENSION AXIS
        a=torch.cat([x,x_m],1)
        return F.relu(a,inplace=True)
    
    def forward(self,x):
        outputs=self._forward(x)
        for i in outputs:
            print(i.shape)
        a=torch.cat(outputs,1)
        print(a.shape)
        a=self.c_relu(a)
        print(a.shape)
        b=self.instance_norm(a)
        print(b.shape)
        b=self.drop(b)
        print(b.shape)
        b=self.pool(b)
        print(b.shape)
        return b
    

In [36]:
class Symbol_Model(nn.Module):
    def __init__(self  ):
        super(Symbol_Model,self).__init__()
        self.conv_block1=Conv_block()
        self.conv_block2=Conv_block(384)
        self.conv1=nn.Conv2d(in_channels=384,out_channels=97,kernel_size=1)
    
    def forward(self,x):
        x=self.conv_block1(x)
        x=self.conv_block2(x)
        x=self.conv1(x)
        print(x.shape)
        x=x.abs_()
        x=x.mean(dim=(2,3))
        x=F.sigmoid(x) 
        print("FINAL TENSOR")
        print(x.shape)
        return x
    
    def loss_fn(output, target):
    loss = torch.exp(torch.sum((output - target)**2))
    return loss
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)                  # Generate predictions
        loss = self.loss_fn(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    # Generate predictions
        loss = self.loss_fn(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

In [37]:
model=Symbol_Model()
# from torchsummary import summary
# summary(model, input_size=(3, 128, 128))
print(model)

Symbol_Model(
  (conv_block1): Conv_block(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1))
    (conv2a): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (conv3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2))
    (instance_norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (drop): Dropout2d(p=0.1, inplace=False)
    (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block2): Conv_block(
    (conv1): Conv2d(384, 64, kernel_size=(3, 3), stride=(1, 1))
    (conv2a): Conv2d(384, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2b): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (conv3): Conv2d(384, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), dilation=(2, 2))
    (instance_norm): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, tr

In [38]:
from PIL import Image
import numpy
image = Image.open("/home/saurabhyadav007/Proj/data/ocr/out/imgs/img17f84856-a511-4ca8-88d0-b6e200edf564.jpeg")
image=numpy.array(image)
import torchvision
image=torchvision.transforms.functional.to_tensor(image)

In [40]:
p=model(image.unsqueeze(0))

torch.Size([1, 64, 170, 89])
torch.Size([1, 64, 170, 89])
torch.Size([1, 64, 170, 89])
torch.Size([1, 192, 170, 89])
torch.Size([1, 384, 170, 89])
torch.Size([1, 384, 170, 89])
torch.Size([1, 384, 170, 89])
torch.Size([1, 384, 85, 44])
torch.Size([1, 64, 83, 42])
torch.Size([1, 64, 83, 42])
torch.Size([1, 64, 83, 42])
torch.Size([1, 192, 83, 42])
torch.Size([1, 384, 83, 42])
torch.Size([1, 384, 83, 42])
torch.Size([1, 384, 83, 42])
torch.Size([1, 384, 41, 21])
torch.Size([1, 96, 41, 21])
FINAL TENSOR
torch.Size([1, 96])


NameError: name 'Model' is not defined

In [ ]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)
    return history